In [1]:
import pandas as pd
import networkx as nx
import itertools

In [2]:
df = pd.read_json("rawfilteredbydate.json")

In [3]:
df.head()

,caption,date
0,The Perlman Music Program Class of 2014,2014-08-22
1,Toby Perlman addresses the crowd,2014-08-22
2,Itzhak Perlman and Catherine Cho perform with...,2014-08-22
3,"Julian Beck, Lyn Barlow, Ted Ayala, Elinor He...",2014-08-22
4,Toby Perlman and guests,2014-08-22


In [4]:
df["caption"] = df['caption'].apply(lambda x: x.replace("\n",""))

In [5]:
df = df[df['caption'].apply(lambda x: len(x)<=250)]

In [159]:
restr = ["The", "Dr", "Verdana", "Helvetica" ,"Chairman","Director",
         "Guests","President","Hospital","Special Surgery","Executive Director"
         ,"New York","Museum","Photographs","Children","The Society","Dinner",
        "Board Member","Castle","President and CEO","Directors","Clockwise","Doubles","BFAnyc","Standing",
        "Susan Magazine","Trustees","Chicago","Yugoslavia","Steering Committee",
        "I Have A Dream"]

In [160]:
prereplace = ["friend","friends","Mayor "]

In [161]:
def replaceWord(s):
    for i in range(len(prereplace)):
        s = s.replace(prereplace[i],"")
    return s

In [162]:
captions = map(lambda x:replaceWord(x),df.caption)

In [163]:
proper_noun_regex = r'([A-Z][A-Za-z-]*(,?\s+(\b(and|von|de|van der|van|der)\b))?(\s[A-Z][A-Za-z\'-]*)*)'

In [164]:
import re

In [165]:
G=nx.MultiGraph()
for caption in captions:
    name = []
    caption_temp  = re.findall(proper_noun_regex,caption)
    for j in range(len(caption_temp)):
        name_temp = caption_temp[j]
        if len(name_temp[0])>5 and len(name_temp[0].split(" ")) > 1 and len(name_temp[0].split(" ")) <=4:
            
            if name_temp[0] == "PatrickMcMullan":
                name.append("Patrick McMullan")
            elif name_temp[0] == "AnnieWatt":
                name.append("Annie Watt")
            else:
                name.append(name_temp[0])
        for word in restr:
            if word in name:
                name.remove(word)
    if len(name) > 2:
        _=[G.add_edge(c[0],c[1],weight=1) for c in itertools.combinations(name,2)]
    elif len(name) > 1:
        G.add_edge(name[0],name[1],weight=1)        

In [166]:
Gg = nx.Graph()
for u,v,data in G.edges_iter(data=True):
    if Gg.has_edge(u,v):
        Gg[u][v]['weight'] += 1
    else:
        Gg.add_edge(u, v, weight=1)

## degrees

In [167]:
degrees=pd.DataFrame(G.degree().values(),index=G.degree().keys()).sort(0,ascending=False)

In [168]:
degrees = degrees.reset_index()

In [169]:
degrees[180:220]

,index,0
180,Michael Cominotto,100
181,Princess Yasmin Aga Khan,99
182,Dorothy Lichtenstein,99
183,Julia Koch,99
184,Jackie Harris Hochberg,99
185,Veronica Kelly,98
186,Mary Alice Stephenson,98
187,Melanie Holland,98
188,Kirk Henckels,98
189,Olivia Chantecaille,97


In [170]:
degrees.columns = ["name","number"]

In [171]:
[i for i in degrees.name if " and " in i]

[u'Alan and Susan Patricof',
 u'John and Margo Catsimatidis',
 u'Chairman and Gala Co-chair',
 u'Justice and Human Rights',
 u'Chairman and CEO',
 u'Shamon, and Diamon Hill',
 u'Cynthia and Dr',
 u'Michael and Eleanora Kennedy',
 u'Kelly, and Dr',
 u'Barbara and Donald Tober',
 u'Absurdistan, and Annie Eggers',
 u'Sex, and Drugs',
 u'Katharine and William Rayner',
 u'Heuvel, and Sally Minard',
 u'Yugoslavia, and Cornelia Guest',
 u'Director and Chief Curator',
 u'Gianni and Daniela Onorato',
 u'Caputo, and Kate Hufft',
 u'Giuseppe and Antonietta Bono',
 u'Alex and Kate Donner',
 u'Castle, and Ingrid Connolly',
 u'John and Aileen Crowley',
 u'Tatiana and Campion Platt',
 u'Ambassador and Mrs',
 u'Grace and Chris Meigher',
 u'Donald and Barbara Tober',
 u'Andrew and Zibby Right',
 u'Dana and Evan Schiff',
 u'Toth, and Drew Perry',
 u'Kathryn and Kenneth Chenault',
 u'Lila and Cokie Rice',
 u'Colonel and Mrs',
 u'Vecchio, and Claudio',
 u'Fish and Wildlife',
 u'Castle, and Dr',
 u'Valenti

In [172]:
degreeres = map(lambda x,y: (x,y),degrees.name[0:100], degrees.number[0:100])

In [173]:
import pickle

In [174]:
with open('degreeres.pickle', 'wb') as f:
    pickle.dump(degreeres, f)

In [14]:
pr=nx.pagerank(Gg)
pranalysis=pd.DataFrame(pr.values(),index=pr.keys()).sort(0,ascending=False)